In [1]:
import os
import sys
from pathlib import Path
PROJECT_DIR = Path.cwd().parent
sys.path.append(str(PROJECT_DIR))

In [2]:
PROJECT_DIR

PosixPath('/home/badrinath.chandana/git/ACharacterInASimulation/adaptive-computation')

In [4]:
from dataset.synthetic_tasks import AddMul, Difficulty

In [5]:
dataset = AddMul(num_samples=10, max_operands=4, max_digits=3, seed=42)

In [9]:
for data in dataset:
    print(data)
    break


(['-', '8', '7', 'add', '6', '8'], ['-', '1', '9'])


In [11]:
difficulty  = Difficulty(max_operands=2, max_digits=1, operations=['mul'])

In [12]:
dataset.set_difficulty(difficulty)

In [16]:
for data in dataset:
    print(data)
    break


(['3', 'mul', '-', '5'], ['-', '1', '5'])
